In [9]:
import csv
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split
%matplotlib inline

# File name
driving_log = "../training-data/driving_log.csv"

# Read data info
rows = []
with open(driving_log) as file:
    reader = csv.reader(file)
    for row in reader:
        rows.append(row)
train_data, valid_data = train_test_split(rows, test_size=0.2)

In [10]:
# Define genorator for training
def generator(data, batch_size = 128):
    data_size = len(data)
    # Loop for ever in generator loop
    while 1:
        shuffle(data)
        for offset in range(0, data_size, batch_size):
            batch = data[offset:offset+batch_size]
            images = []
            mesurements = []
            for sample in batch:
                center_image_file = row[0].split('\\')[-1]
                img = Image.open("../training-data/IMG/"+center_image_file)
                images.append(np.array(img))
                img.close()
                mesurements.append(float(row[3]))
                
            # Convert into np array
            x_train = np.array(images)
            y_train = np.array(mesurements)
            
            yield x_train, y_train

In [11]:
# Set batch size
batch_size=128

# generate train data
train_generator = generator(train_data, batch_size=batch_size)
validation_generator = generator(valid_data, batch_size=batch_size)

In [13]:
from keras.models import Sequential
from keras.layers import Lambda, Flatten, Dense, Cropping2D, Conv2D
from math import ceil

model = Sequential()
model.add(Lambda(lambda x: x/128.0 - 1.0, input_shape=(160,320,3)))
# If tuple of 2 tuples of 2 ints: interpreted as ((top_crop, bottom_crop), (left_crop, right_crop))
model.add(Cropping2D(cropping=((70,25),(0,0))))
model.add(Conv2D(24, (5, 5), activation="relu", strides=(2, 2)))
model.add(Conv2D(36, (5, 5), activation="relu", strides=(2, 2)))
model.add(Conv2D(48, (5, 5), activation="relu", strides=(2, 2)))
model.add(Conv2D(64, (3, 3), activation="relu"))
model.add(Conv2D(64, (3, 3), activation="relu"))
model.add(Flatten())
model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(10))
model.add(Dense(1))

model.compile(loss="mse", optimizer="adam")
model.fit_generator(train_generator, steps_per_epoch=ceil(len(train_data)/batch_size), validation_data=validation_generator,validation_steps=ceil(len(valid_data)/batch_size), shuffle=True, epochs=50, verbose=1)

model.save("model.h5")



Epoch 1/50
279/279 [==============================] - 57s 203ms/step - loss: 4.9113e-04 - val_loss: 1.0023e-16
Epoch 2/50
279/279 [==============================] - 52s 186ms/step - loss: 2.9421e-18 - val_loss: 5.4210e-20
Epoch 3/50
279/279 [==============================] - 52s 188ms/step - loss: 7.8374e-18 - val_loss: 1.7564e-17
Epoch 4/50
279/279 [==============================] - 53s 188ms/step - loss: 1.3597e-16 - val_loss: 3.1486e-15
Epoch 5/50
279/279 [==============================] - 52s 188ms/step - loss: 1.0137e-15 - val_loss: 2.8259e-14
Epoch 6/50
279/279 [==============================] - 52s 185ms/step - loss: 1.6101e-06 - val_loss: 8.1862e-09
Epoch 7/50
279/279 [==============================] - 52s 185ms/step - loss: 2.0525e-10 - val_loss: 2.6563e-18
Epoch 8/50
279/279 [==============================] - 52s 185ms/step - loss: 4.2012e-18 - val_loss: 1.5667e-17
Epoch 9/50
279/279 [==============================] - 52s 186ms/step - loss: 1.8818e-17 - val_loss: 0.0000e+00

In [15]:
from keras.models import load_model
model_load = load_model("model.h5")

images = []
# Use '/' if data collection ran on linux 
center_image_file = row[0].split('\\')[-1]
img = Image.open("../training-data/IMG/"+center_image_file)
images.append(np.array(img))
img.close()

steering_angle = float(model_load.predict(images[0][None, :, :, :], batch_size=1))
print (steering_angle)

7.169061427703127e-05


In [ ]:
images = []
mesurements = []
debug_percent = len(rows)/100
debug_step = 10
debug_count = 0

for row in rows:
    # Use '/' if data collection ran on linux 
    center_image_file = row[0].split('\\')[-1]
    img = Image.open("../training-data/IMG/"+center_image_file)
    images.append(np.array(img))
    img.close()
    mesurements.append(float(row[3]))
    
    if (debug_count > debug_percent*debug_step):
        print (debug_step, "% done")
        debug_step+=10
    debug_count+=1

# Convert into np array
x_train = np.array(images)
y_train = np.array(mesurements)   
print (np.shape(x_train))

# Plot example image
plt.figure()
plt.imshow(images[0])